In [ ]:
from tensorflow.python.keras import Sequential, Input, Model
from tensorflow.python.keras.activations import relu, softmax
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.layers import Dense, Add
from tensorflow.python.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.python.keras.metrics import categorical_accuracy, binary_accuracy
from tensorflow.python.keras.optimizers import adam
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.datasets import cifar10
import numpy as np
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

In [ ]:
def network_builder(network_conf, network_archi):
    
    print("len",len(network_conf))
    
    while len(network_conf) > 0:
        
        # Premiere couche
        if 'input_layer' in network_conf:
            
            # Add first layer
            input_layer = Input(shape=(network_conf['input_layer']['input_dim'],))
            
            # Add first hidden Layer
            hidden1 = Dense(network_conf['hidden_layer']['hidden_dim'],
                           activation=network_conf['hidden_layer']['activation'])(input_layer)
            
            hidden2 = Dense(network_conf['hidden_layer']['hidden_dim'],
                            activation=network_conf['hidden_layer']['activation'])(hidden1)
            
            res = Add()([hidden1, hidden2])
            
            #store data in list
            network_archi.append(input_layer)
            network_archi.append(hidden1)
            network_archi.append(hidden2)
            network_archi.append(res)
            
            
            # Delete pills of stack
            network_conf.pop('input_layer')
            network_builder(network_conf, network_archi)
            
        # Couche caché
        elif len(network_conf) > 1 and network_conf['hidden_layer']['nbr_hidden_layer'] > 0:
            
            hidden = Dense(network_conf['hidden_layer']['hidden_dim'], 
                           activation=network_conf['hidden_layer']['activation'])(network_archi[-1])
            
            res = Add()([network_archi[-2], hidden])
            
            network_archi.append(hidden)
            network_archi.append(res)
            
            # decrement nbr_hidden_layer
            network_conf['hidden_layer']['nbr_hidden_layer'] -= 1
            
            if(network_conf['hidden_layer']['nbr_hidden_layer'] == 0):
                # Derniere couche
                network_conf.pop('hidden_layer')
            
            network_builder(network_conf, network_archi)
        
        elif len(network_conf) == 1:
            output_layer = Dense(network_conf['output_layer']['output_dim'],
                                 activation=network_conf['output_layer']['activation'])(network_archi[-1])
            # print de test
            network_archi.append(output_layer)
            
            #### A VIRER
            print('\n'.join(map(str, network_archi))) 
            
            network_conf.pop('output_layer')
            
            network_archi.append(Model(network_archi[0], output_layer))

In [ ]:
if __name__ == "__main__":
    # Preprocessing
    
    network_conf = {}

    # network_conf = Param
    network_conf.update({'input_layer':{
                                         'input_dim': 3072}
                         })

    network_conf.update({'hidden_layer': {'hidden_dim':128,
                                         'activation':'relu',
                                          'nbr_hidden_layer': 4
                                        }
                         })

    network_conf.update({'output_layer': {'output_dim':10,
                                         'activation':'softmax'}
                          })

    network_archi = []

    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

    X_train = X_train.reshape(50000, 32 * 32 * 3)
    X_test = X_test.reshape(10000, 32 * 32 * 3)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    
    #Param
    nb_epoch = 1
    batch_size = 2048
    nb_classes = 10
    
    
    
    Y_train = np_utils.to_categorical(Y_train, nb_classes)
    Y_test = np_utils.to_categorical(Y_test, nb_classes)
    
    # Call function to build network
    # . network_builder(network_conf, network_archi)
    
    #model = network_builder(network_conf, network_archi)
    #model = network_archi[-1]
    model = create_dense_res_net(3072,128,10) 
    
    model.compile(optimizer=adam(),
                  loss=categorical_crossentropy,
                  metrics=['sparse_categorical_accuracy'])
    
    model.summary()

    tb_callback = TensorBoard(log_dir="res_logs/ResNets_sparse_categorical_accuracy")

    model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
              epochs=nb_epoch, batch_size=512, callbacks=[tb_callback])